In [ ]:
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

data = pd.read_csv(r'E:\Programming\Projects\Python\JSP\classify1.csv')

data = data.sample(frac=1)


In [13]:
from sklearn.model_selection import train_test_split

X = data['review']

y = data['sentiment']

X_train, X_test = train_test_split(X, test_size=0.2)

y_train, y_test = train_test_split(y, test_size=0.2)

In [14]:
y_train.loc[y_train == 'Negative'] = 0
y_train.loc[y_train == 'Positive'] = 1

y_test.loc[y_test == 'Negative'] = 0
y_test.loc[y_test == 'Positive'] = 1


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

vectorizer = TfidfVectorizer()

vectorized_text_train = vectorizer.fit_transform(np.asarray(X_train))


In [16]:
from sklearn import svm

var = svm.SVC(gamma='scale')
var.fit(vectorized_text_train, y_train)
svm.SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
        decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
        max_iter=-1, probability=False, random_state=None, shrinking=True,
        tol=0.001, verbose=False)


In [ ]:
var.predict(y_test)
